In [8]:
import matplotlib.pyplot as plt
import rasterio
from pathlib import Path
import pandas as pd

In [9]:
latex_path = Path("../latex")
latex_path.mkdir(exist_ok=True)

In [10]:
data_path = Path('../data')
cropped_data_path = data_path / 'planet_images_cropped'

df = pd.read_csv(data_path / 'new_validation_table.csv')
df = df[['site_name', 'planet_id', 'dswx_files']]

df.head()

,site_name,planet_id,dswx_files
0,4_21,20210903_150800_60_2458,../data/20210903_150800_60_2458/OPERA_L3_DSWx-...
1,4_11,20210903_152641_60_105c,../data/20210903_152641_60_105c/OPERA_L3_DSWx-...
2,1_31,20210904_093422_44_1065,../data/20210904_093422_44_1065/OPERA_L3_DSWx-...
3,3_28,20210906_101112_28_225a,../data/20210906_101112_28_225a/OPERA_L3_DSWx-...
4,1_37,20210909_000649_94_222b,../data/20210909_000649_94_222b/OPERA_L3_DSWx-...


In [11]:
latex_slide_template = r"""
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
\begin{frame}[fragile]
\frametitle{$<<DSWX_ID>>$}

\begin{center}
\tiny
\begin{verbatim}
DSWx ID: <<DSWX_ID_ORIG>>
SITE ID: <<SITE_ID>>
\end{verbatim}
\includegraphics[scale=0.25]{<<DSWX_DIR>>/chip_extents.png}
\vfill
\input{<<DSWX_DIR>>/requirements.tex}
\vfill
%\input{<<DSWX_DIR>>/requirements_expanded.tex}
\vfill
%\input{<<DSWX_DIR>>/requirements_expanded.tex}
\vfill
\input{<<DSWX_DIR>>/area_summary.tex}
\end{center}
\vfill
\end{frame}
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
\begin{frame}[fragile]
\frametitle{$<<DSWX_ID>>$}

\begin{center}
\tiny
\begin{verbatim}
DSWx ID: <<DSWX_ID_ORIG>>
\end{verbatim}
\vfill
\includegraphics[scale=0.25]{<<DSWX_DIR>>/classifications.png}
\vfill
\end{center}
\end{frame}
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
"""

In [13]:
with open(latex_path / "slides.tex", 'w') as f:
    for i, row in df.iterrows():
        
        planet_id = row.planet_id
        dswx_id_orig = row.dswx_files.split('/')[-1][:-12]
        dswx_id = dswx_id_orig.replace('_', '\_').replace('-', '{\\text -}')

        tmp_text = latex_slide_template.replace("<<DSWX_ID>>", str(dswx_id))
        tmp_text = tmp_text.replace("<<DSWX_ID_ORIG>>", str(dswx_id_orig))
        tmp_text = tmp_text.replace("<<DSWX_DIR>>", str(planet_id))
        tmp_text = tmp_text.replace("<<<Not Passed>>>", "\textcolor{red}{Not Passed}")
        tmp_text = tmp_text.replace("<<PLANET_ID>>", planet_id)
        tmp_text = tmp_text.replace("<<SITE_ID>>", row.site_name)

        f.write(tmp_text)